# [1581. Customer Who Visited but Did Not Make Any Transactions](https://leetcode.com/problems/customer-who-visited-but-did-not-make-any-transactions/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Visits

<pre>+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| visit_id    | int     |
| customer_id | int     |
+-------------+---------+</pre>
visit_id is the column with unique values for this table.
This table contains information about the customers who visited the mall.
 

Table: Transactions

<pre>+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| transaction_id | int     |
| visit_id       | int     |
| amount         | int     |
+----------------+---------+</pre>
transaction_id is column with unique values for this table.
This table contains information about the transactions made during the visit_id.
 

Write a solution to find the IDs of the users who visited without making any transactions and the number of times they made these types of visits.

Return the result table sorted in any order.

The result format is in the following example.

 

Example 1:

Input: 
Visits
<pre>+----------+-------------+
| visit_id | customer_id |
+----------+-------------+
| 1        | 23          |
| 2        | 9           |
| 4        | 30          |
| 5        | 54          |
| 6        | 96          |
| 7        | 54          |
| 8        | 54          |
+----------+-------------+</pre>
Transactions
<pre>+----------------+----------+--------+
| transaction_id | visit_id | amount |
+----------------+----------+--------+
| 2              | 5        | 310    |
| 3              | 5        | 300    |
| 9              | 5        | 200    |
| 12             | 1        | 910    |
| 13             | 2        | 970    |
+----------------+----------+--------+</pre>
Output: 
<pre>+-------------+----------------+
| customer_id | count_no_trans |
+-------------+----------------+
| 54          | 2              |
| 30          | 1              |
| 96          | 1              |
+-------------+----------------+</pre>
Explanation: 
Customer with id = 23 visited the mall once and made one transaction during the visit with id = 12.
Customer with id = 9 visited the mall once and made one transaction during the visit with id = 13.
Customer with id = 30 visited the mall once and did not make any transactions.
Customer with id = 54 visited the mall three times. During 2 visits they did not make any transactions, and during one visit they made 3 transactions.
Customer with id = 96 visited the mall once and did not make any transactions.
As we can see, users with IDs 30 and 96 visited the mall one time without making any transactions. Also, user 54 visited the mall twice and did not make any transactions.

In [1]:
#pandas schema

import pandas as pd

data = [[1, 23], [2, 9], [4, 30], [5, 54], [6, 96], [7, 54], [8, 54]]
visits = pd.DataFrame(data, columns=['visit_id', 'customer_id']).astype({'visit_id':'Int64', 'customer_id':'Int64'})
data = [[2, 5, 310], [3, 5, 300], [9, 5, 200], [12, 1, 910], [13, 2, 970]]
transactions = pd.DataFrame(data, columns=['transaction_id', 'visit_id', 'amount']).astype({'transaction_id':'Int64', 'visit_id':'Int64', 'amount':'Int64'})

# to pyspark schema

#Importing the packages
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

visits_df = spark.createDataFrame(visits)
transactions_df = spark.createDataFrame(transactions)


In [2]:
visits

,visit_id,customer_id
0,1,23
1,2,9
2,4,30
3,5,54
4,6,96
5,7,54
6,8,54


In [4]:

transactions

,transaction_id,visit_id,amount
0,2,5,310
1,3,5,300
2,9,5,200
3,12,1,910
4,13,2,970


In [27]:
# In spark Dataframe
visits_df.join(transactions_df,['visit_id'],'left')\
        .where(col('amount').isNull())\
        .groupBy('customer_id').count().show()
      

+-----------+-----+
|customer_id|count|
+-----------+-----+
|         54|    2|
|         96|    1|
|         30|    1|
+-----------+-----+



In [12]:
# In spark SQL
visits_df.createOrReplaceTempView('visits')
transactions_df.createOrReplaceTempView('transactions')

spark.sql('''
    select customer_id,count(visit_id)
    from visits v left join transactions using(visit_id)
    where amount is NULL
    group by customer_id
''').show()

+-----------+---------------+
|customer_id|count(visit_id)|
+-----------+---------------+
|         54|              2|
|         96|              1|
|         30|              1|
+-----------+---------------+



In [6]:
spark.stop()